#Задание

Реализуйте обучение нейронной сети из одного нейрона для задачи предсказания стоимости квартир [boston house prices](https://www.kaggle.com/vikrishnan/boston-house-prices) или [california housing prices](https://www.kaggle.com/datasets/camnugent/california-housing-prices) с использованием pytorch.

## 1) Загрузите и подготовьте данные

Boston House Prices

Each record in the database describes a Boston suburb or town. The data was drawn from the Boston Standard Metropolitan Statistical Area (SMSA) in 1970. The attributes are deﬁned as follows (taken from the UCI Machine Learning Repository1):

1.CRIM: per capita crime rate by town

2.ZN: proportion of residential land zoned for lots over 25,000 sq.ft.

3.INDUS: proportion of non-retail business acres per town

4.CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)

5.NOX: nitric oxides concentration (parts per 10 million)

6.RM: average number of rooms per dwelling

7.AGE: proportion of owner-occupied units built prior to 1940

8.DIS: weighted distances to ﬁve Boston employment centers

9.RAD: index of accessibility to radial highways

10.TAX: full-value property-tax rate per $10,000

11.PTRATIO: pupil-teacher ratio by town 12.

12.B: 1000(Bk−0.63)2 where Bk is the proportion of blacks by town 13.

13.LSTAT: % lower status of the population

14.MEDV: Median value of owner-occupied homes in \$1000s


In [ ]:
import pandas as pd

In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

In [ ]:
boston_house_prices = pd.read_csv('housing.csv',header=None, delimiter=r"\s+",names=column_names)

In [ ]:
boston_house_prices.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


## 2) Разделите данные на test и train

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
X = boston_house_prices.drop('MEDV', axis=1)

In [ ]:
y = np.array(boston_house_prices.MEDV)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 1)

#3) Создайте класс модели

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
X_train.shape

(404, 13)

In [ ]:
y_train.shape

(404,)

In [ ]:
class TorchRegression():

       def __init__(self, input_size, learning_rate= 0.01, num_epochs= 100):
         self.learning_rate = learning_rate
         self.num_epochs = num_epochs
         self.model = nn.Linear(input_size, 1)
         self.criterion = nn.MSELoss(reduction='mean')
         self.optimizer = optim.SGD(self.model.parameters(), lr=self.learning_rate)

       def fit(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.reshape(-1, 1), dtype=torch.float32)

        for epoch in range(self.num_epochs):

            y_pred = self.model(X_tensor)

            # Вычисление значения функции потерь
            loss = self.criterion(y_pred, y_tensor)

            # Обратное распространение ошибки и оптимизация модели
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if epoch % 100 == 0:
                print(f'Epoch [{epoch}/{self.num_epochs}], Loss: {loss.item():.4f}')

       def predict(self, X):
           with torch.no_grad():
               X_tensor = torch.tensor(X, dtype=torch.float32)
               y_pred = self.model(X_tensor)
               return y_pred.numpy()

## 4) Обучите модель на train данных

In [ ]:
model = TorchRegression(input_size=X.shape[1], learning_rate=0.01, num_epochs=1000)

In [ ]:
model.fit(X_train, y_train)

Epoch [0/1000], Loss: 581.3204
Epoch [100/1000], Loss: 33.4049
Epoch [200/1000], Loss: 23.1174
Epoch [300/1000], Loss: 22.4919
Epoch [400/1000], Loss: 22.2570
Epoch [500/1000], Loss: 22.1256
Epoch [600/1000], Loss: 22.0464
Epoch [700/1000], Loss: 21.9959
Epoch [800/1000], Loss: 21.9621
Epoch [900/1000], Loss: 21.9383


## 5) Проверьте качество модели на тестовых данных

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
test_loss = np.mean((y_test_pred - y_test.reshape(-1, 1))**2)
print(f'Test Loss: {test_loss:.4f}')

Test Loss: 23.4475
